# U-NET FOR PARAMOS IN THE WORLD

In this notebook we will use the U-Net architecture to segment the paramos in the world. 

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext autoreload

### Cuda info 

In [ ]:
!nvidia-smi

In [ ]:

import sagemaker
import os
import boto3
import sys
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
print("Region = {}".format(region))
bucket = sagemaker_session.default_bucket('nombre-de-tu-bucket')
role = sagemaker.get_execution_role()
print("Role = {}".format(role))

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import packages.utils.get_path as path
import config.constants as const
ABSOLUTE_PATH = path.make_path()

In [ ]:
data_train_path = path.make_path(ABSOLUTE_PATH, const.DATA_DIR, const.PROCESSED_DIR, const.TRAIN_DIR)
data_val_path = path.make_path(ABSOLUTE_PATH, const.DATA_DIR, const.PROCESSED_DIR, const.VALIDATION_DIR)
data_eval_path = path.make_path(ABSOLUTE_PATH, const.DATA_DIR, const.PROCESSED_DIR, const.EVALUATION_DIR)
normalized_path = path.make_path(ABSOLUTE_PATH, const.DATA_DIR, const.PROCESSED_DIR, const.NORMALIZATION_DIR)

In [ ]:
import numpy as np

train_rgb_images = np.load("{}/{}".format(data_train_path, const.TRAIN_RGB_NPY))
train_label_images = np.load("{}/{}".format(data_train_path, const.TRAIN_LABEL_NPY))

eval_rgb_images = np.load("{}/{}".format(data_val_path, const.VAL_RGB_NPY))
eval_label_images = np.load("{}/{}".format(data_val_path, const.VAL_LABEL_NPY))

val_rgb_images = np.load("{}/{}".format(data_eval_path, const.EVAL_RGB_NPY))
val_label_images = np.load("{}/{}".format(data_eval_path, const.EVAL_LABEL_NPY))


In [ ]:

#Z-score normalization
mean = np.load("{}/{}".format(normalized_path, const.MEAN_NPY))
std = np.load("{}/{}".format(normalized_path, const.STD_NPY))

train_rgb_images = (train_rgb_images - mean) / std
eval_rgb_images = (eval_rgb_images - mean) / std
val_rgb_images = (val_rgb_images - mean) / std



In [ ]:
# Normalize the masks using the same normalization as the images
train_label_images = train_label_images / 255.0
eval_label_images = eval_label_images / 255.0
val_label_images = val_label_images / 255.0


In [ ]:
# Convert labels to binary masks
train_label_images = np.mean(train_label_images, axis=-1, keepdims=True)
val_label_images = np.mean(val_label_images, axis=-1, keepdims=True)
eval_label_images = np.mean(eval_label_images, axis=-1, keepdims=True)

In [ ]:

# Ensure that the masks are binary (0 or 1)
train_label_images = (train_label_images > 0.5).astype(np.float32)
val_label_images = (val_label_images > 0.5).astype(np.float32)
eval_label_images = (eval_label_images > 0.5).astype(np.float32)

# UNET Model Inference

In [ ]:
# use the model to predict on the test set 

predictions = model.predict(eval_rgb_images)


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Flatten the arrays for sklearn
true_labels = eval_label_images.flatten()
predicted_labels = np.round(predictions).flatten()  # round predictions to 0 or 1

print(classification_report(true_labels, predicted_labels))

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import random
# random numer in eval_label_images  length
index = random.randint(0, len(eval_label_images) - 1) #43 ,482, 596
print(index)
test_image = eval_rgb_images[index]
true_label = eval_label_images[index]
predicted_label = predictions[index]

# Plot the test image and the masks
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(test_image)
axes[0].set_title('Test Image')
axes[1].imshow(true_label.squeeze(), cmap='gray')
axes[1].set_title('True Label')
axes[2].imshow(predicted_label.squeeze(), cmap='gray')
axes[2].set_title('Predicted Label')

html = """
<div style='text-align: center;'>
    <div style='display: inline-block; padding: 10px; background-color: black; color: white; border-radius: 5px; margin: 5px; font-size: 15px; font-weight: bold; box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5); font-family: Arial, sans-serif;'>Black is 'paramo'</div>
    <div style='display: inline-block; padding: 10px; background-color: white; color: black; border: 1px solid black; border-radius: 5px; margin: 5px; font-size: 15px; font-weight: bold; box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5); font-family: Arial, sans-serif;'>White is 'not paramo'</div>
</div>
"""

# Display the HTML snippet
display(HTML(html))
plt.show()